In [1]:
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
import seaborn as sns
data = pd.read_csv('hr.csv')
pd.set_option('display.max_columns', None)

/Users/yang/anaconda3/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2785: DtypeWarning: Columns (48,52,55,56,57,59,60,61,63,64) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [2]:
# transform str to datetime object

dates_cols = ['Birth_Date','PRIMARY_APPT_BEGIN_DATE','ORIGINAL_HIRE_DATE']
for col in dates_cols:
    data[col] = pd.to_datetime(data[col])

not_na = data['TERMINATION_DATE'].notna()
data['TERMINATION_DATE'] = pd.DatetimeIndex(data['TERMINATION_DATE'])

# data['Birth_Date'] = data['Birth_Date'].dt.year
# current_year = datetime.datetime.now().year
# data["Birth_Date"] = data["Birth_Date"].apply(lambda x: current_year - x)
# data = data.rename(columns = {"Birth_Date":'Age'})

In [3]:
# Let's replace "PHILA" with PHILADELPHIA.
data['HOME_CITY'] = data['HOME_CITY'].str.strip()   # gets rid of leading and trailing white space
wrong_philly = data['HOME_CITY'] == 'PHILA'   
data['HOME_CITY'][wrong_philly] = 'PHILADELPHIA'
# data['HOME_CITY'] = data['HOME_CITY'].str.strip()   
# data['HOME_CITY'] = data['HOME_CITY'].apply(lambda x: 'PHILA' if x == 'PHILADELPHIA' or x == 'PHILA' else 'Non-PHILA')

/Users/yang/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  after removing the cwd from sys.path.


In [4]:
data['PAY_CYCLE'] = data['PAY_CYCLE'].apply(lambda x: 0 if x == 'M1' else 1)
data['SEX'] = data['Manager_Sex'].apply(lambda x: 0 if x == 'M' else 1)
data['Manager_Sex'] = data['Manager_Sex'].apply(lambda x: 0 if x == 'M' else 1)

In [5]:
categorical_cols = ['RACE','MARITAL_STATUS','GRANT_PAID','Manager_Race']
for col in categorical_cols:
    df = pd.get_dummies(data[col]).rename(lambda x: col+'_'+str(x), axis=1)
    data = pd.concat([data, df], axis=1)
    data = data.drop(columns = col)

In [6]:
# NOTE: STRONG CORRELATION between
# PRIMARY_APPT_SALARY, Starting_Full_Time_Salary, Career_Salary_Growth, Percent_Career_Salary_Growth
# i.e. PRIMARY_APPT_SALARY = Starting_Full_Time_Salary + Career_Salary_Growth

In [7]:
data['EDUCATION_LEVEL_YEAR'] = data['EDUCATION_LEVEL_YEAR'].str.strip()
data['EDUCATION_LEVEL_YEAR'] = data['EDUCATION_LEVEL_YEAR'].apply(lambda x: int(x) if x.isnumeric() else None)

In [8]:
data['Manager_Edu_Level_Year'] = data['Manager_Edu_Level_Year'].str.strip()
data['Manager_Edu_Level_Year'] = data['Manager_Edu_Level_Year'].fillna('')
data['Manager_Edu_Level_Year'] = data['Manager_Edu_Level_Year'].apply(lambda x: int(x) if x != '' else None)


In [9]:
# average ratings
rating_map = {'E':10,'R':8,'M':6,'N':4,'U':2}
rating_df = pd.DataFrame()
for year in range(2010,2020):
    data['OVR_RATING_CD_'+str(year)] = data['OVR_RATING_CD_'+str(year)].apply(lambda x: rating_map[x] if x in rating_map else 0)
    rating_df = pd.concat([rating_df,data['OVR_RATING_CD_'+str(year)]],axis=1)
    data = data.drop(columns = ['OVR_RATING_DESCR_'+str(year),'OVR_RATING_CD_'+str(year)])

from collections import Counter
valid_years = np.array([10-c[0] for c in [Counter(x) for x in rating_df.values]])
total = np.array(rating_df.apply(np.sum,axis=1).values)
avg_rating = np.zeros(len(total))
for i in range(len(total)):
    if valid_years[i] == 0:
        avg_rating[i] = None
    else:
        avg_rating[i] = total[i]/valid_years[i]
data['avg_rating'] = pd.Series(avg_rating)

In [10]:
redundant_cols = ['Race_Desc','Sex_Desc','EDUCATION_LEVEL_DESC','HOME_ZIP','YEARS_IN_PRIMARY_JOB','YEARS_OF_SERVICE',
                  'EMPLOYMENT_STATUS_DESC',
                  'Dept_Name','Dept_Name_Short',
                  'School_Name','School_Abbrev',
                  'PRIMARY_APPT_JOB_CLASS',
                  'Job_Grade_Fiscal_Year','AnnualMinimum',
                  'AnnualFirstThird','AnnualMidPoint','AnnualSecondThird','AnnualMax',
                  'REPORTS_TO_PENN_ID',
                  'REPORTS_TO_DATE',
                  'Manager_Years_Managed',
                  'Manager_Original_Hire_Date',
                  'Manager_Edu_Level_Desc','Manager_Sex_Desc','Manager_Race_Desc','Termination_Fiscal_Year',
                  'SEPERATION_REASON_DESC',
                  'Retirement_Eligible_Date']

data = data.drop(columns = redundant_cols)

In [11]:
text_cols = ['EDUCATION_LEVEL','COUNTRY_OF_CITIZENSHIP','HOME_CITY','VISA','JOB_GRADE','JOB_TITLE','Manager_Edu_Level']
for col in text_cols:
    data[col] = data[col].str.strip()

In [12]:
display(data)

,Worker_ID,Birth_Date,Race_OfColor,SEX,AGE_AT_TERMINATION,CURRENT_AGE,EDUCATION_LEVEL,EDUCATION_LEVEL_YEAR,COUNTRY_OF_CITIZENSHIP,HOME_CITY,VISA,VISA_EXPIRATION_DATE,PRIMARY_APPT_BEGIN_DATE,MONTHS_IN_PRIMARY_JOB,HOME_DEPT_ORG,HOME_SCHOOL_CTR,ORIGINAL_HIRE_DATE,PRIMARY_APPT_SALARY,JOB_GRADE,JOB_TITLE,IN_UNION,PAY_CYCLE,Starting_Full_Time_Salary,Career_Salary_Growth,Percent_Career_Salary_Growth,Number_of_Sal_Adjustments,Number_of_Reclass,Manager_Months_Managed,Manager_Current_Age,Manager_Age_At_Termination,Manager_Edu_Level,Manager_Edu_Level_Year,Manager_YEARS_OF_SERVICE,Manager_Sex,Manager_OfColor,Manager_Job_Title,TERMINATION_DATE,EMPLOYMENT_STATUS,SEPARATION_REASON,VOLUNTARY,JOBS_ACTIVE,RACE_A,RACE_B,RACE_C,RACE_P,RACE_R,RACE_S,RACE_U,RACE_Z,MARITAL_STATUS_D,MARITAL_STATUS_M,MARITAL_STATUS_S,MARITAL_STATUS_W,GRANT_PAID_F,GRANT_PAID_N,GRANT_PAID_P,Manager_Race_A,Manager_Race_B,Manager_Race_C,Manager_Race_P,Manager_Race_R,Manager_Race_S,Manager_Race_U,Manager_Race_Z,avg_rating
0,1,1982-07-13,0,0,28.0,37,C,NaN,,NEWTOWN,,NaN,2008-09-08,32,4613,40,2008-09-08,41615.00,25,TECH PRESSURE CHAMBER,0,1,40000.00,1615.00,0.0403,0,0,3.0,91.0,83.0,H,1954.0,56.0,0,0.0,ASSOC DIRECTOR D,2011-05-09,T,24,0.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,4.000000
1,2,1984-02-10,0,0,27.0,35,G,2006.0,,PHILADELPHIA,,NaN,2011-01-03,10,4413,40,2007-07-27,32714.00,25,CLINICAL RESEARCH COORD A,0,0,26000.00,6714.00,0.2582,0,0,9.0,71.0,64.0,,NaN,4.0,0,0.0,MGR RES PROJECT A,2011-11-21,T,41,1.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0,7.000000
2,3,1965-03-26,0,1,46.0,54,G,1995.0,,PHILADELPHIA,,NaN,2004-12-03,84,225,2,2004-12-03,62636.72,B,PROGRAMMER ANALYST,0,0,52000.00,10636.72,0.2045,0,0,10.0,71.0,63.0,K,NaN,12.0,1,0.0,LIMITED SERVICE (E),2011-12-30,T,1,0.0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,10.000000
3,4,1959-09-30,0,1,52.0,60,G,1983.0,,HADDON HEIGHT,,NaN,2002-07-01,115,4212,40,1987-02-16,90815.00,29,ADMIN & FINANCIAL OFFICER,0,0,65000.00,25815.00,0.3971,0,0,12.0,72.0,64.0,H,1976.0,28.0,1,0.0,HR DIR SCHOOL/CENTER,2012-02-17,T,80,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,8.000000
4,5,1990-02-12,0,0,23.0,29,,NaN,,PHILADELPHIA,,NaN,2013-01-01,4,120,2,2011-09-06,29120.00,24,RESEARCH SPECIALIST A,0,0,29120.00,0.00,0.0000,0,0,3.0,86.0,79.0,K,1960.0,31.0,0,0.0,EMERITUS PROFESSOR,2013-05-24,T,47,1.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,NaN
5,6,1985-10-21,0,0,25.0,33,,NaN,,NEWTOWNSQUARE,,NaN,2009-03-09,26,4202,40,2009-03-09,27895.49,24,RESEARCH SPECIALIST A,0,0,27000.00,895.49,0.0331,0,0,3.0,65.0,56.0,I,1980.0,17.0,0,0.0,CPUP CLINICAL POSITION,2011-05-31,T,47,1.0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,8.000000
6,7,1980-05-04,0,0,31.0,39,,NaN,,VOORHEES,,NaN,2008-01-22,49,4413,40,2008-01-22,41593.00,26,CLINICAL RESEARCH COORD B,0,0,23313.00,18280.00,0.7841,0,0,12.0,70.0,62.0,I,1975.0,32.0,0,0.0,CPUP CLINICAL POSITION,2012-02-24,T,41,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,0,1,0,0,0,0,0,10.000000
7,8,1978-08-29,0,1,32.0,41,G,2008.0,,SYRACUSE,,NaN,2010-01-04,18,5110,51,2010-01-04,33014.00,24,RESEARCH SPECIALIST A,0,0,31500.00,1514.00,0.0480,0,0,3.0,74.0,66.0,K,1982.0,24.0,1,0.0,PROFESSOR A,2011-07-28,T,47,1.0,0,0,0,1,0,0,0,0,0,0,0,1,0,1,0,0,0,0,1,0,0,0,0,0,8.000000
8,9,1980-05-23,0,1,30.0,39,H,2009.0,,PHILADELPHIA,,NaN,2009-09-08,19,603,6,2009-08-10,53170.00,27,MGR RES PROJECT A,0,0,52000.00,1170.00,0.0225,0,0,2.0,64.0,55.0,K,1987.0,16.0,1,1.0,PROFESSOR A,2011-04-09,T,41,1.0,0,0,0,1,0,0,0,0,0,0,1,0,0,1,0,0,0,1,0,0,0,0,0,0,10.000000
9,10,1972-01-10,1,1,41.0,47,H,2004.0,,PHILADELPHIA,,NaN,2009-11-16,40,603,6,2009-11-16,54695.56,28,MGR RES PROJECT B,0,0,50000.00,4695.56,0.0939,0,0,24.0,64.0,57.0,K,1987.0,17.0,1,1.0,PROFESSOR A,2013-03-01,T,41,1.0,0,0,1,0,0,0,0,0,0,0,0,1,0,0,0,1,0,1,0,0,0,0,0,0,10.000000


In [13]:
data.to_csv('clean_data.csv',index=False, date_format='%Y-%m-%d')